In [1]:
#pip install -r requirements.txt

# Pacotes

In [1]:
# Importações da biblioteca padrão
import os
import time
 
# Importações de terceiros para raspagem de dados na web
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup

# Manipulação de dados
import pandas as pd

# Trabalhar com arquivos zip
import zipfile

from concurrent.futures import ThreadPoolExecutor

In [2]:
# Bibliotecas padrão
import os
import time

# Bibliotecas de terceiros para raspagem de dados na web
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import bs4
from bs4 import BeautifulSoup

# Manipulação de dados
import pandas as pd

# Trabalhar com arquivos zip
import zipfile

from concurrent.futures import ThreadPoolExecutor

# Dicionário com as bibliotecas e seus métodos de versão
libraries = {
    "os": os.__name__,
    "time": time.__name__,
    "requests": requests.__version__,
    "bs4": bs4.__version__,  # A versão vem do módulo bs4, não de BeautifulSoup
    "pandas": pd.__version__,
    "zipfile": "Built-in library (version included in Python)",
    "concurrent.futures": "Built-in library (version included in Python)"
}

# Printando a versão de cada biblioteca
for lib, version in libraries.items():
    print(f"{lib}: {version}")


os: os
time: time
requests: 2.31.0
bs4: 4.12.2
pandas: 2.1.4
zipfile: Built-in library (version included in Python)
concurrent.futures: Built-in library (version included in Python)


# Funções

In [7]:
def download_csv_files(year):
    """
    Faz o download de arquivos CSV de dados históricos de voos para o ano especificado a partir do site da ANAC.

    Args:
    year (int): O ano para o qual os dados devem ser baixados.

    Etapas:
    1. Configurar uma sessão de requests com regras de nova tentativa em caso de falhas.
    2. Realizar uma requisição GET para a URL base e usar BeautifulSoup para analisar o conteúdo HTML.
    3. Criar um diretório para o ano especificado, se não existir.
    4. Mapear os meses em português para códigos numéricos e encontrar a seção correspondente ao ano desejado.
    5. Iterar sobre os meses, localizar os links de download dos arquivos CSV, e fazer o download de cada arquivo.
    6. Em caso de falha no download, registrar a exceção.
    7. Pausar por 7 segundos entre os downloads para evitar sobrecarga do servidor.

    Returns:
    None. Os arquivos são salvos localmente.
    """
    mouth = ["1","2","3","4","5","6","7","8","9", "10", "11","12"]
    # Definição da URL base para os dados climáticos do ano especificado.
    base_url = f"https://portal.inmet.gov.br/uploads/dadoshistoricos/{year}/VRA_{year}_{mouth}.csv"

    # Configuração de uma sessão HTTP com regras de retry para lidar com falhas de conexão ou timeouts.
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    # Requisição GET para a URL base.
    response = session.get(base_url)
    # Uso de BeautifulSoup para parsear a resposta HTML.
    soup = BeautifulSoup(response.content, 'html.parser')

    # Criação do diretório para o ano especificado, se não existir.
    os.makedirs(str(year), exist_ok=True)

    # Mapeamento dos meses em português para o formato numérico padrão.
    month_mapping = {
        'Janeiro': '01', 'Fevereiro': '02', 'Março': '03', 'Abril': '04', 'Maio': '05', 'Junho': '06',
        'Julho': '07', 'Agosto': '08', 'Setembro': '09', 'Outubro': '10', 'Novembro': '11', 'Dezembro': '12'
    }

    # Localização da seção do ano especificado no HTML.
    year_section = soup.find('h2', string=str(year))
    if not year_section:
        print(f"Ano {year} não encontrado na página.")
        return

    # Iteração sobre os meses para encontrar e fazer download dos arquivos CSV.
    for month, month_code in month_mapping.items():
        link = year_section.find_next('a', string=month, href=True)
        if link:
            file_url = link['href']
            local_file_name = f"{year}/vra-{month_code}_{year}.csv"
            try:
                # Requisição GET para o arquivo CSV e gravação do conteúdo no arquivo local.
                file_response = session.get(file_url)
                file_response.raise_for_status()
                with open(local_file_name, 'wb') as file:
                    file.write(file_response.content)
                print(f"Downloaded {local_file_name}")
            except requests.exceptions.RequestException as e:
                print(f"Failed to download {file_url}: {e}")
        else:
            print(f"Link para {month} não encontrado.")

        # Pausa entre downloads para não sobrecarregar o servidor.
        time.sleep(10)

def download_dados_complementares(file_to_download=None):
    """
    Faz o download e a conversão de arquivos complementares relacionados a dados de voos.
    
    Etapas:
    1. Definir as URLs dos arquivos para download e os nomes dos arquivos.
    2. Criar diretórios para armazenar os arquivos baixados e convertidos, se não existirem.
    3. Fazer o download de cada arquivo .xls e salvar localmente.
    4. Converter cada arquivo .xls baixado em um arquivo .csv, ajustando as colunas conforme necessário.
    5. Em caso de falha no download ou na conversão, registrar a exceção.
    
    Args:
    file_to_download: Nome opcional do arquivo específico a ser baixado. Se None, todos os arquivos serão baixados.
    
    Returns:
    None. Os arquivos são salvos e convertidos localmente.
    """
    
    # Definição das URLs dos arquivos de dados complementares e seus respectivos nomes.
    urls = {
        "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/vra/glossario_de_aerodromo.xls": "glossario_de_aerodromo",
        "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/vra/glossario_de_empresas_aereas.xls": "glossario_de_empresas_aereas",
        "https://sistemas.anac.gov.br/dadosabertos/Aerodromos/Aeródromos%20Públicos/Lista%20de%20aeródromos%20públicos/AerodromosPublicos.csv": "aerodromos_brasil"
    }

    # Filtrar o dicionário de URLs se um arquivo específico for requisitado.
    if file_to_download:
        urls = {url: name for url, name in urls.items() if name == file_to_download}

    # Caminhos para os diretórios onde os arquivos serão salvos.
    dir_path = "dados_complementares"

    # Criação do diretório, se ele não existir.
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    # Processo de download e conversão dos arquivos.
    for url, file_name in urls.items():
        # Caminho do arquivo a ser baixado.
        file_path = os.path.join(dir_path, f"{file_name}")
        response = requests.get(url)
        if response.status_code == 200:
            # Verifica a extensão do arquivo para decidir o processo de salvamento.
            if url.lower().endswith('.xls'):
                # Se o arquivo é um .xls, salva e converte para .csv.
                file_path += '.xls'
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {file_path}")

                # Tentativa de conversão do arquivo .xls para .csv.
                try:
                    # Carregamento e limpeza dos dados do arquivo .xls.
                    df = pd.read_excel(file_path, header=3)
                    df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)

                    # Caminho do arquivo .csv convertido.
                    csv_path = file_path.replace('.xls', '.csv')
                    # Salvando os dados limpos em formato .csv.
                    df.to_csv(csv_path, index=False)
                    print(f"Converted to {csv_path}")
                except Exception as e:
                    print(f"Error converting {file_path} to CSV: {e}")
            elif url.lower().endswith('.csv'):
                # Se o arquivo já é um .csv, apenas salva.
                file_path += '.csv'
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {file_path}")
        else:
            print(f"Failed to download {url}")


def download_and_extract_climate_data(year):
    """
    Realiza o download de arquivos ZIP de dados climáticos de um ano específico a partir do portal do INMET
    e extrai os conteúdos para uma pasta nomeada 'CLIMA_<ANO>'.

    Args:
    year (int): O ano dos dados climáticos a serem baixados.

    Etapas:
    1. Configurar uma sessão de requests com regras de retry para lidar com falhas de conexão ou timeouts.
    2. Construir a URL do arquivo ZIP para o ano especificado.
    3. Criar um diretório para armazenar os dados desse ano, se não existir.
    4. Realizar o download do arquivo ZIP.
    5. Extrair o conteúdo do arquivo ZIP no diretório criado.
    6. Lidar com falhas no download ou na extração, registrando exceções.
    7. Aguardar um intervalo entre downloads para evitar sobrecarga do servidor.

    Returns:
    None. Os arquivos são baixados e extraídos localmente.
    """
    
    # Definição da URL base para os dados climáticos do ano especificado.
    base_url = f"https://portal.inmet.gov.br/uploads/dadoshistoricos/{year}.zip"

    # Configuração de sessão HTTP com políticas de retry.
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    # Criação do diretório para armazenar os dados do ano.
    directory_name = f"DADOS_METEROLOGIA/CLIMA_{year}"
    os.makedirs(directory_name, exist_ok=True)

    # Nome do arquivo ZIP local.
    local_zip_file = f"{directory_name}/{year}.zip"
    
    try:
        # Download do arquivo ZIP.
        print(f"Iniciando download do arquivo {year}.zip...")
        zip_response = session.get(base_url)
        zip_response.raise_for_status()

        # Salvar o conteúdo do ZIP no arquivo local.
        with open(local_zip_file, 'wb') as file:
            file.write(zip_response.content)
        print(f"Download do arquivo {year}.zip concluído.")

        # Extração do conteúdo do arquivo ZIP.
        with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
            zip_ref.extractall(directory_name)
        print(f"Arquivos extraídos para a pasta {directory_name}.")

    except requests.exceptions.RequestException as e:
        print(f"Falha no download do arquivo {year}.zip: {e}")

    except zipfile.BadZipFile as e:
        print(f"Falha ao extrair o arquivo {year}.zip: {e}")

    # Remoção do arquivo ZIP após a extração.
    os.remove(local_zip_file)
    print(f"Arquivo ZIP {local_zip_file} removido.")

    # Intervalo entre downloads para evitar sobrecarga do servidor.
    time.sleep(7)

# Extração

In [8]:
def download_csv_files(year):
    """
    Faz o download dos arquivos CSV de dados históricos de voos da ANAC para um ano específico.
    
    Args:
        year (int): Ano desejado para download dos dados.
    
    Etapas:
    1. Configura a sessão HTTP com política de retries.
    2. Define a URL base e gera URLs corretas para cada mês.
    3. Cria diretório local para armazenar os arquivos.
    4. Faz o download de cada arquivo mensal.
    5. Trata exceções e aguarda entre downloads.
    
    Returns:
        None. Arquivos salvos localmente.
    """

    # URL base para downloads (ANAC)
    base_url = f"https://siros.anac.gov.br/siros/registros/diversos/vra/{year}/"

    # Sessão HTTP com retries configurados
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=2, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    session.mount('http://', HTTPAdapter(max_retries=retries))

    # Criação do diretório para o ano especificado
    directory = str(year)
    os.makedirs(directory, exist_ok=True)

    # Iteração pelos 12 meses
    for month in range(1, 13):
        month_code = str(month).zfill(2)  # Formato '01', '02', ..., '12'
        file_name = f"VRA_{year}_{month_code}.csv"
        file_url = f"{base_url}{file_name}"
        local_file_path = os.path.join(directory, file_name)

        try:
            response = session.get(file_url)
            response.raise_for_status()  # Garante que houve sucesso no download
            
            # Salva o arquivo localmente
            with open(local_file_path, 'wb') as file:
                file.write(response.content)

            print(f"Downloaded: {local_file_path}")

        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar {file_url}: {e}")

        # Aguarda 7 segundos para evitar sobrecarga do servidor
        time.sleep(7)

In [9]:
# Exemplo de uso:
download_csv_files(2024)

Downloaded: 2024\VRA_2024_01.csv
Downloaded: 2024\VRA_2024_02.csv
Downloaded: 2024\VRA_2024_03.csv
Downloaded: 2024\VRA_2024_04.csv
Downloaded: 2024\VRA_2024_05.csv
Downloaded: 2024\VRA_2024_06.csv
Downloaded: 2024\VRA_2024_07.csv
Downloaded: 2024\VRA_2024_08.csv
Downloaded: 2024\VRA_2024_09.csv
Downloaded: 2024\VRA_2024_10.csv
Downloaded: 2024\VRA_2024_11.csv
Downloaded: 2024\VRA_2024_12.csv


In [3]:
# Chamar a função para o ano de 2018
download_csv_files(2023)

Downloaded 2023/vra-01_2023.csv
Downloaded 2023/vra-02_2023.csv
Downloaded 2023/vra-03_2023.csv
Downloaded 2023/vra-04_2023.csv
Downloaded 2023/vra-05_2023.csv
Downloaded 2023/vra-06_2023.csv
Downloaded 2023/vra-07_2023.csv
Downloaded 2023/vra-08_2023.csv
Downloaded 2023/vra-09_2023.csv
Downloaded 2023/vra-10_2023.csv
Downloaded 2023/vra-11_2023.csv
Downloaded 2023/vra-12_2023.csv


In [4]:
# Executar a função
download_dados_complementares()

Downloaded dados_complementares\glossario_de_aerodromo.xls
Converted to dados_complementares\glossario_de_aerodromo.csv
Downloaded dados_complementares\glossario_de_empresas_aereas.xls
Converted to dados_complementares\glossario_de_empresas_aereas.csv
Downloaded dados_complementares\aerodromos_brasil.csv


In [10]:
# Exemplo de uso da função
download_and_extract_climate_data(2024)  # Altere para o ano desejado.

Iniciando download do arquivo 2024.zip...
Download do arquivo 2024.zip concluído.
Arquivos extraídos para a pasta DADOS_METEROLOGIA/CLIMA_2024.
Arquivo ZIP DADOS_METEROLOGIA/CLIMA_2024/2024.zip removido.


# Utilizando as funções acima para extrair mais de um ano

In [5]:
# Lista dos anos para os quais você deseja baixar os dados
years = [2018, 2019, 2020,2021,2022,2023]

# Usando ThreadPoolExecutor para baixar arquivos CSV de dados históricos de voos
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(download_csv_files, years)

Downloaded 2018/vra-01_2018.csv
Downloaded 2020/vra-01_2020.csv
Downloaded 2019/vra-01_2019.csv
Downloaded 2018/vra-02_2018.csv
Downloaded 2019/vra-02_2019.csv
Downloaded 2020/vra-02_2020.csv
Downloaded 2018/vra-03_2018.csv
Downloaded 2019/vra-03_2019.csv
Downloaded 2020/vra-03_2020.csv
Downloaded 2018/vra-04_2018.csv
Downloaded 2019/vra-04_2019.csv
Downloaded 2020/vra-04_2020.csv
Downloaded 2018/vra-05_2018.csv
Downloaded 2020/vra-05_2020.csv
Downloaded 2019/vra-05_2019.csv
Failed to download https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_062018.csv: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
Downloaded 2020/vra-06_2020.csv
Downloaded 2019/vra-06_2019.csv
Downloaded 2020/vra-07_2020.csv
Downloaded 2018/vra-07_2018.csv
Downloaded 2019/vra-07_2019.csv
Downloaded 2020/vra-08_2020.csv
Downloaded 2018/vra-08_2018.csv
Downloaded 2019/vra-08_2019.csv
D

In [6]:
# Usando ThreadPoolExecutor para baixar e extrair dados climáticos
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(download_and_extract_climate_data, years)

Iniciando download do arquivo 2018.zip...
Iniciando download do arquivo 2019.zip...
Iniciando download do arquivo 2020.zip...
Download do arquivo 2018.zip concluído.
Arquivos extraídos para a pasta DADOS_METEROLOGIA/CLIMA_2018.
Arquivo ZIP DADOS_METEROLOGIA/CLIMA_2018/2018.zip removido.
Iniciando download do arquivo 2021.zip...
Download do arquivo 2020.zip concluído.
Arquivos extraídos para a pasta DADOS_METEROLOGIA/CLIMA_2020.
Arquivo ZIP DADOS_METEROLOGIA/CLIMA_2020/2020.zip removido.
Iniciando download do arquivo 2022.zip...
Download do arquivo 2021.zip concluído.
Arquivos extraídos para a pasta DADOS_METEROLOGIA/CLIMA_2021.
Arquivo ZIP DADOS_METEROLOGIA/CLIMA_2021/2021.zip removido.
Download do arquivo 2019.zip concluído.
Arquivos extraídos para a pasta DADOS_METEROLOGIA/CLIMA_2019.
Arquivo ZIP DADOS_METEROLOGIA/CLIMA_2019/2019.zip removido.
Iniciando download do arquivo 2023.zip...
Download do arquivo 2022.zip concluído.
Arquivos extraídos para a pasta DADOS_METEROLOGIA/CLIMA_2022